In [1]:
#uses DotProduct_#N_S#noepsE_HRx#hrf_C#mamcs_A#baseAdderSize - v4.5

In [2]:
import sys
import math

In [3]:
#M>=1
M = input('Enter M: ')

Enter M: 1


In [4]:
#N>=2
N = input('Enter N: ')

Enter N: 2


In [5]:
#>=1, <=M
PR = input('Enter No Of Parallel Row (empty for 1): ')

Enter No Of Parallel Row (empty for 1): 1


In [6]:
M = int(M)
if(M<1):
    M=1
    
N = int(N)
if(N<2):
    N=2

if not PR:
    PR=1
else:
    PR = int(PR)
    if PR>M:
        PR = M

In [7]:
NOEPS = input('Enter No Of Elements Per Series (>=1, <={}, empty for {}): '.format(N-1, N-1))

Enter No Of Elements Per Series (>=1, <=1, empty for 1): 1


In [8]:
if not NOEPS:
    NMult=N-1
else:
    NMult = int(NOEPS)
    if NMult<1:
        NMult=1
    elif NMult>(N-1):
        NMult=N-1

In [9]:
#MAMCS>=1
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(NMult))

Enter Multiply-Add Max Component Size (empty for 1): 1


In [10]:
if not MAMCS:
    MAMCS = NMult
else:
    MAMCS = int(MAMCS)
    if MAMCS>NMult:
        MAMCS = NMult;
    elif(MAMCS<1):
            MAMCS=1

In [11]:
#AdderSize>=2
if MAMCS<NMult:
    AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

In [12]:
RS=math.ceil(M/PR)

In [13]:
if RS>1:
    Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')

In [14]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 


In [15]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')

Enter INPUT_REG_DEPTH default value (empty for 1): 


In [16]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')

Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [17]:
HRR = math.ceil(N/NMult);

NumOfComponents = math.ceil(NMult/MAMCS)
if MAMCS==NMult:
    AdderSize=0
else:
    if not AdderSize:
        AdderSize=2
    else:
        AdderSize = int(AdderSize)
        if AdderSize<2:
            AdderSize=2
        if AdderSize>NumOfComponents:
            AdderSize=NumOfComponents

if RS==1:
    Enable_Vector_Latch = 0;
else:
    if not Enable_Vector_Latch:
        Enable_Vector_Latch = 0
    else:
        Enable_Vector_Latch = int(Enable_Vector_Latch)

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [18]:
ModuleName="MatrixVectorMultiply_{}_{}_{}PR_S{}E_HRx{}_C{}_A{}_".format(M, N, PR, NMult, HRR, MAMCS, AdderSize)
if Enable_Vector_Latch<=0:
    ModuleName+="N"
ModuleName+="VL"

In [19]:
__Print_To_File = 0

if __Print_To_File<=0:
    of = sys.stdout
else:
    of = open("./"+ModuleName+".v", 'w+')

In [20]:
print("`timescale 1ns / 1ps\n", file=of)
print("module "+ModuleName, file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("output newInSeriesStart,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output inSeries,", file=of)
elif lghrr>1:
    print("output [{}:0] inSeries,".format(lghrr-1), file=of)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("output reg rowSetInNo = 0,", file=of)
else:
    print("output reg [{}:0] rowSetInNo = 0,".format(lgRS-1), file=of)
print("input inReady,", file=of)
for j in range(PR):
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult):
        print("AE{}R{}, ".format(i, j), end='', file=of)
    print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NMult):
    print("VE{}, ".format(i), end='', file=of)
print(file=of)
ar=RS*PR-M
ME = PR-ar-1
if ME==0:
    st1="E0"
else:
    st1="E0toE{}".format(ME)
print("output ", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUCReady = 0, //not used", file=of)
else:
    if ME==(PR-2):
        st2="E{}".format(PR-1)
    else:
        st2="E{}toE{}".format(ME+1, PR-1)
    print("output ", end='', file=of)
    print(st2, end='', file=of)
    print("Ready,", file=of)
if lgRS<=1:
    print("output reg rowSetOutNo = {},".format(RS-1), file=of)
else:
    print("output reg [{}:0] rowSetOutNo = {},".format(lgRS-1, RS-1), file=of)
lgn=math.ceil(math.log2(N))
al=-1+lgn
if al<0:
    print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
elif al==0:
    print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
for i in range(PR):
    print("E{}, ".format(i), end='',file=of)
print(file=of)
print("output early", end='', file=of)
print(st1, end='', file=of)
print("Ready,", file=of)
if ar==0:
    print("output reg NUearlyCReady = 0 //not used", file=of)
else:
    print("output early", end='', file=of)
    print(st2, end='', file=of)
    print("Ready", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module MatrixVectorMultiply_1_2_1PR_S1E_HRx2_C1_A0_NVL
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
output newInSeriesStart,
output inSeries,
output reg rowSetInNo = 0,
input inReady,
input signed [IN_WIDTH-1:0] AE0R0, 
input signed [IN_WIDTH-1:0] VE0, 
output E0Ready,
output reg NUCReady = 0, //not used
output reg rowSetOutNo = 0,
output signed [(2*IN_WIDTH):0] E0, 
output earlyE0Ready,
output reg NUearlyCReady = 0 //not used
);



In [21]:
if Enable_Vector_Latch>0:
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult-1):
        print("SVE{}, ".format(i), end='', file=of)
    print("SVE{};".format(NMult-1), file=of)
    print(file=of)

In [22]:
if(ar>0): #RS*PR > M, RS>=2
    print("wire inReadyC = (rowSetInNo!={}) & inReady;".format(RS-1), file=of)
    print(file=of)
for i in range (PR):
    print("DotProduct_{}_S{}E_HRx{}_C{}_A{}".format(N, NMult, HRR, MAMCS, AdderSize), file=of)
    print("#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
    print("DP_{}(clk, reset, enable,".format(i), file=of)
    if i==0:
        print("newInSeriesStart,", file=of)
        print("inSeries,".format(i), file=of)
    else:
        print("newInSeriesStart{}, //not used".format(i), file=of)
        print("inSeries{}, //not used".format(i), file=of)
    if(PR-i<=ar):
        print("inReadyC,", file=of)
    else:
        print("inReady,", file=of)
    for j in range(NMult):
        print("AE{}R{}, ".format(j, i), end='', file=of)
    print(file=of)
    if Enable_Vector_Latch>0:
        for j in range(NMult):
            print("SVE{}, ".format(j), end='', file=of)
    else:
        for j in range(NMult):
            print("VE{}, ".format(j), end='', file=of)        
    print(file=of)
    if i==0:
        print(st1+"Ready,", file=of)
    elif i==PR-ar:
        print(st2+"Ready,", file=of)
    else:
        print("outReady{}, //not used".format(i), file=of)
    print("E{},".format(i), file=of)
    if i==0:
        print("early"+st1+"Ready);", file=of)
    elif i==PR-ar:
        print("early"+st2+"Ready);", file=of)
    else:
        print("earlyOutReady{}); //not used".format(i), file=of)
    print(file=of)


DotProduct_2_S1E_HRx2_C1_A0
#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
DP_0(clk, reset, enable,
newInSeriesStart,
inSeries,
inReady,
AE0R0, 
VE0, 
E0Ready,
E0,
earlyE0Ready);



In [23]:
if(RS>1):
    if Enable_Vector_Latch>0:
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(N-1):
            print("LVE{}, ".format(i), end='', file=of)
        print("LVE{};".format(N-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetInNo <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    print("\t\t\tif(inSeries=={}) begin".format(HRR-1), file=of)
    print("\t\t\t\tif(rowSetInNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\t\trowSetInNo <= 0;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\t\trowSetInNo <= 1;", file=of)    
    else:
        print("\t\t\t\t\trowSetInNo <= rowSetInNo+1;", file=of)
    print("\t\t\t\tend", file=of)
    print("\t\t\tend", file=of)
    if Enable_Vector_Latch>0:
        print("\t\t\tif(rowSetInNo==0) begin", file=of)
        print("\t\t\t\tcase(inSeries)", file=of)
        for i in range(HRR):
            print("\t\t\t\t\t{}: begin".format(i), file=of)
            print("\t\t\t\t\t\t", end='', file=of)
            k=NMult*i-1
            for j in range(NMult):
                k=k+1
                print("LVE{} <= VE{}; ".format(k,j), end='', file=of)
                if k == N-1:
                    break
            print(file=of)
            print("\t\t\t\t\tend", file=of)
        print("\t\t\t\tendcase", file=of)
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [24]:
if Enable_Vector_Latch>0:
    print("always @(*) begin", file=of)
    print("\tif(rowSetInNo==0) begin", file=of)
    print("\t\t", end='', file=of)
    for j in range(NMult):
        print("SVE{} = VE{}; ".format(j,j), end='', file=of)
    print(file=of)
    print("\tend", file=of)
    print("\telse begin", file=of)
    print("\t\tcase(inSeries)", file=of)
    for i in range(HRR):
        print("\t\t\t{}: begin".format(i), file=of)
        print("\t\t\t\t", end='', file=of)
        k=NMult*i-1
        for j in range(NMult):
            k=k+1
            if k >= N:
                print("SVE{} = 0; ".format(j), end='', file=of)            
            else:
                print("SVE{} = LVE{}; ".format(j,k), end='', file=of)
        print(file=of)
        print("\t\t\tend", file=of)        
    if (HRR & (HRR-1)) != 0: #check if HRR is not power of 2
        print("\t\t\tdefault: begin".format(k), file=of)
        print("\t\t\t\t", end='', file=of)
        for j in range(NMult):
            print("SVE{} = 0; ".format(j), end='', file=of)
        print(file=of)
        print("\t\t\tend", file=of)        
    print("\t\tendcase", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [25]:
if(RS>1):
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\trowSetOutNo <= {};".format(RS-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(early"+st1+"Ready) begin", file=of)
    print("\t\t\tif(rowSetOutNo=={}) begin".format(RS-1), file=of)
    print("\t\t\t\trowSetOutNo <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if RS==2:
        print("\t\t\t\trowSetOutNo <= 1;", file=of)
    else:
        print("\t\t\t\trowSetOutNo <= rowSetOutNo+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

In [26]:
print("endmodule", file=of)

endmodule


In [27]:
if __Print_To_File>0:
    of.close()